# Lab 4: Adding Trajectory Evaluations


In this lab, you will compute the convergence score of the agent, and you will use Phoenix experiments to set up this evaluation. The video goes through the components of an experiment (dataset consisting of examples, task, evaluator). 

Here's how you will define the experiment components in this lab to compute the convergence score:

<img src="images/experiment1.png" width="700"/>

You will create a dataset of Phoenix examples where each example contains a different version of the same question. You will then create the task `run_agent_and_track_path` that you will use to run on each example and compute the path length. Finally, you will create the evaluator `evaluate_path_length` that computes the convergence score.

## Importing necessary libraries 

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from datetime import datetime

import nest_asyncio
import pandas as pd
import phoenix as px
from phoenix.experiments import evaluate_experiment, run_experiment
from phoenix.experiments.evaluators import create_evaluator
from phoenix.experiments.types import Example

nest_asyncio.apply()

In [3]:
from utils import get_phoenix_endpoint, run_agent

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: evaluating-agent
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.comv1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [4]:
get_phoenix_endpoint()

'https://app.phoenix.arize.com'

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code>, <code>utils.py</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Your results might differ from those shown in the video.</p>

## Creating the Dataset of Test Cases

In [5]:
px_client = px.Client()
help(px_client)

Help on Client in module phoenix.session.client object:

class Client(phoenix.session.data_extractor.TraceDataExtractor)
 |  Client(*, endpoint: Optional[str] = None, warn_if_server_not_running: bool = True, headers: Optional[collections.abc.Mapping[str, str]] = None, api_key: Optional[str] = None, **kwargs: Any)
 |  
 |  Method resolution order:
 |      Client
 |      phoenix.session.data_extractor.TraceDataExtractor
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *, endpoint: Optional[str] = None, warn_if_server_not_running: bool = True, headers: Optional[collections.abc.Mapping[str, str]] = None, api_key: Optional[str] = None, **kwargs: Any)
 |      Client for connecting to a Phoenix server.
 |      
 |      Args:
 |          endpoint (str, optional): Phoenix server endpoint, e.g.
 |              http://localhost:6006. If not provided, the endpoint will be
 |              inferred from the environment variables.
 |      
 |          hea

In [6]:
convergence_questions = [
    "What was the average quantity sold per transaction?",
    "What is the mean number of items per sale?", 
    "Calculate the typical quantity per transaction",
    "What's the mean transaction size in terms of quantity?",
    "On average, how many items were purchased per transaction?",
    "What is the average basket size per sale?",
    "Calculate the mean number of products per purchase",
    "What's the typical number of units per order?",
    "What is the average number of products bought per purchase?",
    "Tell me the mean quantity of items in a typical transaction",
    "How many items does a customer buy on average per transaction?",
    "What's the usual number of units in each sale?",
    "What is the typical amount of products per transaction?",
    "Show the mean number of items customers purchase per visit",
    "What's the average quantity of units per shopping trip?",
    "How many products do customers typically buy in one transaction?",
    "What is the standard basket size in terms of quantity?"
]

convergence_df = pd.DataFrame({
    'question': convergence_questions
})

now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
dataset = px_client.upload_dataset(dataframe=convergence_df, 
                                   dataset_name=f"convergence_questions-{now}",
                                   input_keys=["question"])

📤 Uploading dataset...
💾 Examples uploaded: https://app.phoenix.arize.com/datasets/RGF0YXNldDoz/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246Mw==


### Link to Phoenix UI

You can open this link to check out the Phoenix UI and the uploaded dataset. You can use the same link to check out the results of the experiment you'll run in this notebook. 

**Note**: 
- Since each notebook of this course runs in an isolated environment, each notebook links to a different Phoenix server. This is why you won't see the projects you've worked on in the previous notebooks. 
- Make sure that the notebook's kernel is running when checking the Phoenix UI. If the link does not open, it might be because the notebook has been open or inactive for a long time. In that case, make sure to refresh the browser, run all previous cells and then check this link. 

In [7]:
print(get_phoenix_endpoint())

https://app.phoenix.arize.com


## Creating the Task

In [8]:
# helper method to format the output returned by the task
def format_message_steps(messages):
    """
    Convert a list of message objects into a readable format that shows the steps taken.

    Args:
        messages (list): A list of message objects containing role, content, tool calls, etc.

    Returns:
        str: A readable string showing the steps taken.
    """
    steps = []
    for message in messages:
        role = message.get("role")
        if role == "user":
            steps.append(f"User: {message.get('content')}")
        elif role == "system":
            steps.append("System: Provided context")
        elif role == "assistant":
            if message.get("tool_calls"):
                for tool_call in message["tool_calls"]:
                    tool_name = tool_call["function"]["name"]
                    steps.append(f"Assistant: Called tool '{tool_name}'")
            else:
                steps.append(f"Assistant: {message.get('content')}")
        elif role == "tool":
            steps.append(f"Tool response: {message.get('content')}")
    
    return "\n".join(steps)

In [9]:
def run_agent_and_track_path(example: Example) -> str:
    messages = [{"role": "user", "content": example.input.get("question")}]
    ret = run_agent(messages)
    return {"path_length": len(ret), "messages": format_message_steps(ret)}

## Running the Experiment

In [10]:
experiment = run_experiment(dataset,
                            run_agent_and_track_path,
                            experiment_name="Convergence Eval",
                            experiment_description="Evaluating the convergence of the agent")

🧪 Experiment started.
📺 View dataset experiments: https://app.phoenix.arize.com/datasets/RGF0YXNldDoz/experiments
🔗 View this experiment: https://app.phoenix.arize.com/datasets/RGF0YXNldDoz/compare?experimentId=RXhwZXJpbWVudDox


running tasks |          | 0/17 (0.0%) | ⏳ 00:00<? | ?it/s

Running agent with messages: [{'role': 'user', 'content': 'What was the average quantity sold per transaction?'}]
Starting router call span


Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openinference/instrumentation/openai/_request.py", line 297, in __call__
    response = wrapped(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 919, in request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 1008, in _request
    return self._retry_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 1057, in _retry_request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-servic

Traceback (most recent call last):
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/phoenix/experiments/functions.py", line 305, in async_run_experiment
    _output = task(*bound_task_args.args, **bound_task_args.kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/80/_4jfr9sn6m3_8lbtyt_p2fm80000gn/T/ipykernel_67366/3347942733.py", line 3, in run_agent_and_track_path
    ret = run_agent(messages)
          ^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/arize_evals/utils.py", line 330, in run_agent
    response = client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_utils/_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platfor

...

Traceback (most recent call last):
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/phoenix/experiments/functions.py", line 305, in async_run_experiment
    _output = task(*bound_task_args.args, **bound_task_args.kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/80/_4jfr9sn6m3_8lbtyt_p2fm80000gn/T/ipykernel_67366/3347942733.py", line 3, in run_agent_and_track_path
    ret = run_agent(messages)
          ^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/arize_evals/utils.py", line 330, in run_agent
    response = client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_utils/_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platfor

Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openinference/instrumentation/openai/_request.py", line 297, in __call__
    response = wrapped(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 919, in request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 1008, in _request
    return self._retry_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 1057, in _retry_request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-servic

Traceback (most recent call last):
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/phoenix/experiments/functions.py", line 305, in async_run_experiment
    _output = task(*bound_task_args.args, **bound_task_args.kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/80/_4jfr9sn6m3_8lbtyt_p2fm80000gn/T/ipykernel_67366/3347942733.py", line 3, in run_agent_and_track_path
    ret = run_agent(messages)
          ^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/arize_evals/utils.py", line 330, in run_agent
    response = client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_utils/_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platfor

Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openinference/instrumentation/openai/_request.py", line 297, in __call__
    response = wrapped(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 919, in request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 1008, in _request
    return self._retry_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_base_client.py", line 1057, in _retry_request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-servic

Traceback (most recent call last):
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/phoenix/experiments/functions.py", line 305, in async_run_experiment
    _output = task(*bound_task_args.args, **bound_task_args.kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/80/_4jfr9sn6m3_8lbtyt_p2fm80000gn/T/ipykernel_67366/3347942733.py", line 3, in run_agent_and_track_path
    ret = run_agent(messages)
          ^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/arize_evals/utils.py", line 330, in run_agent
    response = client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platform-service/venv/lib/python3.11/site-packages/openai/_utils/_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/thongbuiwork/sondermind/assistant-platfor

In [11]:
experiment.as_dataframe()

,error,input,example_id
run_id,,,
RXhwZXJpbWVudFJ1bjox,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'What was the average quantity so...,RGF0YXNldEV4YW1wbGU6MTE4
RXhwZXJpbWVudFJ1bjoy,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'What is the mean number of items...,RGF0YXNldEV4YW1wbGU6MTE5
RXhwZXJpbWVudFJ1bjoz,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'Calculate the typical quantity p...,RGF0YXNldEV4YW1wbGU6MTIw
RXhwZXJpbWVudFJ1bjo0,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'What's the mean transaction size...,RGF0YXNldEV4YW1wbGU6MTIx
RXhwZXJpbWVudFJ1bjo1,"RateLimitError(""Error code: 429 - {'error': {'...","{'question': 'On average, how many items were ...",RGF0YXNldEV4YW1wbGU6MTIy
RXhwZXJpbWVudFJ1bjo2,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'What is the average basket size ...,RGF0YXNldEV4YW1wbGU6MTIz
RXhwZXJpbWVudFJ1bjo3,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'Calculate the mean number of pro...,RGF0YXNldEV4YW1wbGU6MTI0
RXhwZXJpbWVudFJ1bjo4,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'What's the typical number of uni...,RGF0YXNldEV4YW1wbGU6MTI1
RXhwZXJpbWVudFJ1bjo5,"RateLimitError(""Error code: 429 - {'error': {'...",{'question': 'What is the average number of pr...,RGF0YXNldEV4YW1wbGU6MTI2


## Evaluating the Path

In [12]:
outputs = experiment.as_dataframe()["output"].to_dict().values()

# Will include the user and system messages
optimal_path_length = min(output.get('path_length') for output in outputs if output and output.get('path_length') is not None)
print(f"The optimal path length is {optimal_path_length}")

KeyError: 'output'

In [ ]:
@create_evaluator(name="Convergence Eval", kind="CODE")
def evaluate_path_length(output: str) -> float:
    if output and output.get("path_length"):
        return optimal_path_length/float(output.get("path_length"))
    else:
        return 0

In [ ]:
experiment = evaluate_experiment(experiment,
                            evaluators=[evaluate_path_length])

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>